In [ ]:
# Importing the required library
import sys
import os 
import gym
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch import optim
import numpy as np
import random
import math
from torch.utils.tensorboard import SummaryWriter
import time 
import argparse
sys.argv = ['']
del sys



In [ ]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("-v_1", "--verbose_1", help=" ",
                    action="store_true", default='Original+linear layer to see if Q is negative ' )
parser.add_argument("-seed", "--seed", help="increase output verbosity",
                    action="store_true",default = 1)
parser.add_argument("-NUMBER_OF_EPISODES", "--NUMBER_OF_EPISODES", help="Total_Episodes to run ",
                    action="store_true",default = 200)
parser.add_argument("-MAX_STEPS", "--MAX_STEPS", help="MAX_STEPS to run each episode ",
                    action="store_true",default = 1369)
parser.add_argument("-LEARNING_RATE", "--LEARNING_RATE", help="LEARNING_RATE ",
                    action="store_true",default = 0.0001) #0.0001

parser.add_argument("-UCB_LEARNING_RATE", "--UCB_LEARNING_RATE", help="UCB_LEARNING_RATE ",
                    action="store_true",default = 0.0001) #0.0001

parser.add_argument("-UCB_FILTER", "--UCB_FILTER", help="UCB_FILTER ",
                    action="store_true",default = 0.0001)

parser.add_argument("-DISCOUNT_FACTOR", "--DISCOUNT_FACTOR", help="DISCOUNT_FACTOR ",
                    action="store_true",default = 0.99)

parser.add_argument("-HIDDEN_LAYER_SIZE", "--HIDDEN_LAYER_SIZE", help="HIDDEN_LAYER_SIZE ", 
                    action="store_true",default = 128)

parser.add_argument("-UCB_HIDDEN_LAYER_SIZE", "--UCB_HIDDEN_LAYER_SIZE", help="UCB HIDDEN_LAYER_SIZE ",
                    action="store_true", default=128)

parser.add_argument("-EGREEDY_START", "--EGREEDY_START", help="EGREEDY_START ",
                    action="store_true",default = 0.4) #0.4
parser.add_argument("-EGREEDY_FINAL", "--EGREEDY_FINAL", help="EGREEDY_FINAL ",
                    action="store_true",default = 0.01) #0.01
parser.add_argument("-EGREEDY_DECAY", "--EGREEDY_DECAY", help="EGREEDY_DECAY- not being used ",
                    action="store_true",default = 2000)
parser.add_argument("-REPLAY_BUFFER_SIZE", "--REPLAY_BUFFER_SIZE", help="REPLAY_BUFFER_SIZE ",
                    action="store_true",default = 40000)
parser.add_argument("-BATCH_SIZE", "--BATCH_SIZE", help="BATCH_SIZE ",
                    action="store_true",default = 64)
parser.add_argument("-UPDATE_TARGET_FREQUENCY", "--UPDATE_TARGET_FREQUENCY", help="UPDATE_TARGET_FREQUENCY ",
                    action="store_true", default=2000)
parser.add_argument("-TEST_MODEL_FREQ", "--TEST_MODEL_FREQ", type=int, help="After How many episodes you want to test the model ",
                     default=1)

parser.add_argument("-TEST_RUN_EPISODE", "-TEST_RUN_EPISODE", help=" Number of episodes to run while testing . ",
                    action="store_true", default=10)


parser.add_argument("-MODEL_SAVE_FREQUENCY", "--MODEL_SAVE_FREQUENCY", help="After How many episodes you want to save the model ",
                    action="store_true", default=10)

parser.add_argument("-TRAINING_STARTS", "--TRAINING_STARTS", type=int, help="After How many episodes does the optimization starts,currently 10 cycles. ",
                     default=0)

parser.add_argument("-TAU", "--TAU", type=int, help="Target Network Update.,tau = 0 no update , tau = 1 imediate update of target with latest, new_target = q * tau + (1-tau) * target ",
                     default=0.5)




parser.add_argument("-run_name", "--run_name", help="run name",
                    action="store_true", default=f"{'_vehcile_13_random_initial_SOC'}_{int(time.time())}")
args = parser.parse_args()
# args = argparse.Namespace(verbose=False, verbose_1=False)


In [ ]:
writer = SummaryWriter(f"runs/{'DDQN'}{args.run_name}")


In [ ]:
# selecting the available device (cpu/gpu)

use_cuda = torch.cuda.is_available()

device = torch.device("cuda:0" if use_cuda else "cpu")

print(device)

In [ ]:
# Creating the CartPole-v0 environment

env = gym.make('CartPole-v0')
env.reset()
# Creating the CartPole-v0 environment

# env = gym.make('CartPole-v0')
# env = gym.make('Vehicle-fixedSOC')
env.reset()
action = env.action_space.sample()
new_state, reward, done, info = env.step(action)
# new_state[2]



In [ ]:
# setting the seed value for reproducibility

# seed = 1626
env.seed(args.seed)
torch.manual_seed(args.seed)
random.seed(args.seed)


In [ ]:
# run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
# run_name = f"{seed}__{int(time.time())}"


In [ ]:
# Fetching the number of states and actions
#number_of_states = env.observation_space.n
number_of_states = env.observation_space.shape[0]
number_of_actions = env.action_space.n
# checking the total number of states and action
print('Total number of States : {}'.format(number_of_states)) 
print('Total number of Actions : {}'.format(number_of_actions))

In [ ]:
def linear_schedule(EGREEDY: float, EGREEDY_FINAL: float, duration: int, t: int):
    slope = (EGREEDY_FINAL - EGREEDY) / duration
    return max(slope * t + EGREEDY, EGREEDY_FINAL)


In [ ]:
# Deep Q Network Model Architecture

class DQN(nn.Module):
    def __init__(self , hidden_layer_size):
        super().__init__()
        torch.manual_seed(args.seed)
        self.hidden_layer_size = hidden_layer_size
        self.fc1 = nn.Linear(number_of_states, self.hidden_layer_size)
        self.fc2 = nn.Linear(self.hidden_layer_size,number_of_actions)
        
    # def _init_weights(self, module):
    #     if isinstance(module, nn.Linear):
    #         module.weight.data.normal_(mean=0.0, std=config.initializer_range)
    #         if module.bias is not None:
    #             module.bias.data.zero_()

    def forward(self, x):
        # x = self.fc0(x)
        output = torch.relu(self.fc1(x))
        output = self.fc2(output)
        return output

In [ ]:
class ExperienceReplay(object):
    def __init__(self , capacity):
        
        self.capacity = capacity
        self.buffer = []
        self.pointer = 0
    
    def push(self , state, action, new_state, reward, done):
        experience = (state, action, new_state, reward, done)
        
        if self.pointer >= len(self.buffer):
            self.buffer.append(experience)
        else:
            self.buffer[self.pointer] = experience
        
        self.pointer = (self.pointer + 1) % self.capacity
        
    def sample(self , batch_size):
        # random.seed(args.seed)
        return zip(*random.sample(self.buffer , batch_size))
    
    def __len__(self):
        return len(self.buffer)    

In [ ]:
# Instantiating the ExperienceReplay
memory = ExperienceReplay(args.REPLAY_BUFFER_SIZE)


In [ ]:
# Building the brain of the network i.e. the DQN Agent

class DQN_Agent(object):
    def __init__(self):
        torch.manual_seed(args.seed)
        self.dqn = DQN(args.HIDDEN_LAYER_SIZE).to(device)
        self.target_dqn = DQN(args.HIDDEN_LAYER_SIZE).to(device)
        self.target_dqn.load_state_dict(self.dqn.state_dict())
        
        # self.criterion = torch.nn.MSELoss()
        self.criterion = torch.nn.SmoothL1Loss(size_average=None, reduce=None, reduction='mean', beta=5.0)
        
        self.optimizer = optim.Adam(
            params=self.dqn.parameters(), lr=args.LEARNING_RATE)
        
        self.target_dqn_update_counter = 0
        self.best_reward = -99999
        self.prev_best_episode = 0
    
    def select_action(self, state, EGREEDY):
        
        random_for_egreedy = torch.rand(1)[0]
        
        if random_for_egreedy > EGREEDY:      
            
            with torch.no_grad():
                
                state = torch.Tensor(state).to(device)
                q_values = self.dqn(state) # + round(random.uniform(0.1, 5), 1)
                action = torch.max(q_values,0)[1]
                action = action.item()        
        else:
            # random.seed(args.seed)
            action = env.action_space.sample()
            # action = 1998
            # print(action)
            
        # print(action)
        return action
    
    def optimize(self,epi,LR):
        self.optimizer = optim.Adam(
            params=self.dqn.parameters(), lr=LR)
        
        if (args.BATCH_SIZE > len(memory)):
            return
        
        state, action, new_state, reward, done = memory.sample(args.BATCH_SIZE)
        # print(reward)
        
        state = torch.Tensor(state).to(device)
        new_state = torch.Tensor(new_state).to(device)
        reward = torch.Tensor(reward).to(device)
        action = torch.LongTensor(action).to(device)
        done = torch.Tensor(done).to(device)
        
        # select action : get the index associated with max q value from prediction network
        new_state_indxs = self.dqn(new_state).detach() 
        max_new_state_indxs = torch.max(new_state_indxs, 1)[1] # to get the max new state indexes
        # print(max_new_state_indxs)
        
        
        # Using the best action from the prediction nn get the max new state value in target dqn
        new_state_values = self.target_dqn(new_state).detach()
        max_new_state_values = new_state_values.gather(1, max_new_state_indxs.unsqueeze(1)).squeeze(1)
        # print(max_new_state_values)
        
        target_value = reward + (1 - done) * args.DISCOUNT_FACTOR * max_new_state_values #when done = 1 then target = reward
        
        predicted_value = self.dqn(state).gather(1, action.unsqueeze(1)).squeeze(1)
        # print()
        # print(predicted_value,target_value)
        loss = self.criterion(predicted_value, target_value)
        # loss = self.criterion(target_value, predicted_value)
        
        # following is for logging only##################
        loss_output = self.criterion(predicted_value, target_value)
        loss_output_diff = predicted_value - target_value
       ############################################### ####
        writer.add_scalar("train/losses/td_loss_steps_abs", loss_output_diff.mean().item(), epi)
        writer.add_scalar("train/losses/td_loss_steps", loss.mean().item(), epi)
        writer.add_scalar("train/losses/predictedmean", predicted_value.mean().item(), epi)
        writer.add_scalar("train/losses/targetmean", target_value.mean().item(), epi)
        writer.add_scalar("train/losses/predictedmin", predicted_value.min().item(), epi)
        writer.add_scalar("train/losses/targetmin", target_value.min().item(), epi)
        writer.add_scalar("train/losses/predictedmax",
                          predicted_value.max().item(), epi)
        writer.add_scalar("train/losses/targetmax", target_value.max().item(), epi)
    
        self.optimizer.zero_grad()
        loss.backward()
        for tag, value in self.dqn.named_parameters():
            writer.add_scalar("train/charts/gradsum ",
                            np.array(value.grad.cpu()).sum(), epi)
            writer.add_scalar("train/charts/gradmean", np.array(value.grad.cpu()).mean(), epi)
            writer.add_scalar("train/charts/gradmin", np.array(value.grad.cpu()).min(), epi)
            writer.add_scalar("train/charts/gradmax", np.array(value.grad.cpu()).max(), epi)
        self.optimizer.step()
        
        if (self.target_dqn_update_counter % args.UPDATE_TARGET_FREQUENCY == 0):
            # self.target_dqn.load_state_dict(self.dqn.state_dict())
            ## new
            target_net_state_dict = self.target_dqn.state_dict()
            policy_net_state_dict = self.dqn.state_dict()
            for key in policy_net_state_dict:
                target_net_state_dict[key] = policy_net_state_dict[key] * \
                args.TAU + target_net_state_dict[key]*(1-args.TAU)
            self.target_dqn.load_state_dict(target_net_state_dict)
            
        return (loss.mean().item())
        
    def save_model(self,epi,total_epi):
        if (epi % args.MODEL_SAVE_FREQUENCY ==0):
            torch.save(self.dqn.state_dict(),
                       f"runs/{'DDQN'}{args.run_name}/{'DDQN'}{args.run_name}_{'episode#'}_{epi}.pt")
            

In [ ]:
# Instantiating the DQN Agent
torch.manual_seed(args.seed)
dqn_agent = DQN_Agent()
# torch.manual_seed(args.seed)
model_dqn = dqn_agent.dqn
# writer = SummaryWriter(f"runs/{'DDQN'}{run_name}")

In [ ]:
# Write the hyperparameters to tensorboard
writer.add_text(
    "hyperparameters",
    "|param|value|\n|-|-|\n%s" % (
        "\n".join([f"|{key}|{value}|" for key, value in vars(args).items()]))
)


In [ ]:
import numpy as np
steps_total = []
reward_total_all_episodes = []
steps_counter = 0
episode_counter = []
e_counter = 1
episode_done = False
loss_step_total = []
q_pred_step_total=[]
q_targ_step_total=[]
for episode in range(args.NUMBER_OF_EPISODES+1):
    print('###############Running Trainning#############################')
    env = gym.make('Vehicle-v0') 
    
    episode_counter.append(episode)
    if episode == args.NUMBER_OF_EPISODES:
        episode_done = True
        
    state = env.reset()
    done = False
    # print('episode :',e_counter)
    LR_comp = linear_schedule(
        args.LEARNING_RATE, args.LEARNING_RATE/20, args.NUMBER_OF_EPISODES, e_counter)
    writer.add_scalar("train/learning_rate", LR_comp, episode)

    step = 0
    reward_total = []
    reward_app = []
    # EGREEDY = calculate_epsilon(NUMBER_OF_EPISODES)
    if episode<args.NUMBER_OF_EPISODES-10:
        EGREEDY = linear_schedule(args.EGREEDY_START, args.EGREEDY_FINAL, args.NUMBER_OF_EPISODES, e_counter)
        # LR_comp = linear_schedule(
        #     args.LEARNING_RATE, 0.0001, args.NUMBER_OF_EPISODES, e_counter)
    else:
        EGREEDY = linear_schedule(
            args.EGREEDY_START, args.EGREEDY_FINAL, args.NUMBER_OF_EPISODES, e_counter)
        # LR_comp = linear_schedule(
        #     args.LEARNING_RATE, 0.0001, args.NUMBER_OF_EPISODES, e_counter)
    print('Epsilon:', EGREEDY)
    print('Episode:',e_counter)
   
    for i in range(args.MAX_STEPS):
        step += 1
        steps_counter += 1
        
        [state1,state2,state3] = state
        writer.add_scalar("train/states/state1", state1, steps_counter)
        writer.add_scalar("train/states/state2", state2, steps_counter)
        writer.add_scalar("train/states/state3", state3, steps_counter)
        
        writer.add_scalar("train/state1", state1, step)
        writer.add_scalar("train/state2", state2, step)
        writer.add_scalar("train/state3", state3, step)
            
        action = dqn_agent.select_action(state, EGREEDY)
            
        writer.add_scalar("train/action/action", action, steps_counter)
            
        new_state, reward, done, info = env.step(action)
        memory.push(state, action, new_state, reward, done)
        reward_app.append(reward)
        if(episode > args.TRAINING_STARTS):
            loss_step = dqn_agent.optimize(steps_counter,LR_comp)
            if loss_step == None:
                loss_step = 0
                q_pred_step = 0
                q_targ_step = 0
            loss_step_total.append(loss_step)
            # q_pred_step_total.append(q_pred_step)
            # q_targ_step_total.append(q_targ_step)
            state = new_state
            # os.system('cls')
            # UU = ucb_agent.UCB_calc(1, 1.0, state, episode)
            writer.add_scalar("train/charts/epsilon", EGREEDY, episode)
            if done:
                steps_total.append(step)
                reward_total = sum(reward_app)
                writer.add_scalar("train/reward", reward_total, episode)
                reward_total_all_episodes.append(reward_total)
                print('loss',sum(loss_step_total)/len(loss_step_total))
                writer.add_scalar("train/losses/td_loss",sum(loss_step_total)/len(loss_step_total), episode)
                e_counter = e_counter+1
                dqn_agent.save_model(episode, args.NUMBER_OF_EPISODES)
                
            